In [6]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """
    Function connects to a database.
    
    Parameters
    ----------
    db_file : rString
        File path to the name of the database you would like to connect to.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()
            
if __name__ == '__main__':
    create_connection(r"C:\sqlite\db\weather.db")

2.6.0


In [2]:
def create_connection(db_file):
    """
    Function connects to a database.
    
    Parameters
    ----------
    db_file : rString
        File path to the name of the database you would like to connect to.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return conn

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
    
def main():
    database = r"C:\sqlite\db\weather.db"
    
    sql_create_weather_table = """ CREATE TABLE IF NOT EXISTS weather_table (
                                    id integer PRIMARY KEY,
                                    city_name text NOT NULL,
                                    state_name text,
                                    datetime text,
                                    temperature float 
                                    ); """
    
    conn = create_connection(database)
    
    if conn is not None:
        create_table(conn, sql_create_weather_table)
    else:
        print("Error! Cannot create the database connection!")
            
if __name__ == '__main__':
    main()

In [7]:
import requests

def create_connection(db_file):
    """
    Function connects to a database.
    
    Parameters
    ----------
    db_file : rString
        File path to the name of the database you would like to connect to.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
        
    return conn

def create_project(conn, project):
    sql = ''' INSERT INTO weather_table(city_name, state_name, datetime, temperature)
                VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

In [ ]:
import sqlite3
from sqlite3 import Error
import requests
import time
import json
import datetime

d = datetime.datetime

database = r"C:\sqlite\db\weather.db"
conn = create_connection(database)

austin_url = "https://api.openweathermap.org/data/2.5/weather?q=Austin&appid=e9d2f749c8d866a3f1aa783791b18cdb&units=imperial"
san_francisco_url = "https://api.openweathermap.org/data/2.5/weather?q=San%20Francisco&appid=e9d2f749c8d866a3f1aa783791b18cdb&units=imperial"
new_york_url = "https://api.openweathermap.org/data/2.5/weather?q=New%20York&appid=e9d2f749c8d866a3f1aa783791b18cdb&units=imperial"

central_time_adjustment = 10800
pacific_time_adjustment = 3600
eastern_time_adjustment = 14400


def create_connection(db_file):
    """
    Function connects to a SQLite database.
    
    Parameters
    ----------
    db_file : rString
        File path to the name of the database you would like to connect to.
        
    Return
    ------
    Connection obj or None.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
        
    return conn

def create_project(conn, project):
    """
    Insert new values into the weather_table
    
    Parameters
    ----------
    conn : conn
    project : string, string, datetime, float
        weather_table values as city_name, state_name, datetime, and temperature
        
    Returns
    -------
    Values into raw text file and inserts into SQLite weather_table
    """
    sql = ''' INSERT INTO weather_table(city_name, state_name, datetime, temperature)
                VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

class City:
    def __init__(self, city_name, state_name, time_adjustment):
        self.city_name = city_name
        self.state_name = state_name
        self.time_adjustment = time_adjustment
        
    def make_url(self, city_name):
        return f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid=e9d2f749c8d866a3f1aa783791b18cdb&units=imperial".format(city_name = city_name)
    
    def request_api(self):
        pass
    
    def write_raw(self):
        pass
austin = City("Austin", "Texas", central_time_adjustment)

while True:
    austin_response = requests.get(austin_url)
    san_francisco_response = requests.get(san_francisco_url)
    new_york_response = requests.get(new_york_url)

    with open('raw_weather.txt', 'a') as f:
        f.write(json.dumps(austin_response.json()))
        f.write(json.dumps(san_francisco_response.json()))
        f.write(json.dumps(new_york_response.json()))

    with conn:
        austin_weather_data = (austin_response.json()['name'],
                               "Texas",
                               d.fromtimestamp(austin_response.json()['dt'] + central_time_adjustment),
                               austin_response.json()['main']['temp'])
        san_francisco_weather_data = (san_francisco_response.json()['name'],
                                      "California",
                                      d.fromtimestamp(san_francisco_response.json()['dt'] + pacific_time_adjustment),
                                      san_francisco_response.json()['main']['temp'])
        new_york_weather_data = (new_york_response.json()['name'],
                                 "New York",
                                 d.fromtimestamp(new_york_response.json()['dt'] + eastern_time_adjustment),
                                 new_york_response.json()['main']['temp'])

        create_project(conn, austin_weather_data)
        create_project(conn, san_francisco_weather_data)
        create_project(conn, new_york_weather_data)

    time.sleep(300)

In [8]:
database = r"C:\sqlite\db\weather.db"
conn = create_connection(database)
sql = 'DELETE FROM weather_table'
cur = conn.cursor()
cur.execute(sql)
conn.commit()

In [53]:
import requests
import time
import json
import datetime
import pytz

d = datetime.datetime
aust_tz = pytz.timezone('US/Central')
sf_tz = pytz.timezone('US/Pacific')
ny_tz = pytz.timezone('US/Eastern')
print(d.now(aust_tz))
print(d.now(sf_tz))
print(d.now(ny_tz))

appid = "e9d2f749c8d866a3f1aa783791b18cdb"

ausLat = 30.2672
ausLon = -97.7431

sfLat = 37.7749
sfLon = -122.4194

nyLat = 40.7128
nyLon = -74.0060
aust_tz = pytz.timezone('US/Central')
sf_tz = pytz.timezone('US/Pacific')
ny_tz = pytz.timezone('US/Eastern')

response = requests.get("https://api.openweathermap.org/data/2.5/weather?q=San%20Francisco&appid=e9d2f749c8d866a3f1aa783791b18cdb"
                        .format(apiKey=appid))
aust_response = json.dumps(response.json(), indent=4, sort_keys=True)
print(aust_response)

SyntaxError: invalid syntax (Temp/ipykernel_6420/1463412320.py, line 31)

In [28]:
import sqlite3
from sqlite3 import Error
import requests
import time
import json
import datetime

d = datetime.datetime
database = r"C:\sqlite\db\weather.db"

central_time_adjustment = 10800
pacific_time_adjustment = 3600
eastern_time_adjustment = 14400

def create_connection(db_file):
    """
    Function connects to a SQLite database.
    
    Parameters
    ----------
    db_file : String
        File path of the database you would like to connect to.
        
    Return
    ------
    Connection obj or None.
    """
    
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
        
    return conn

def create_project(conn, project):
    """
    Insert new values into the weather_table.
    
    Parameters
    ----------
    conn : conn
    project : string, string, datetime, float
        weather_table values as city_name, state_name, datetime, and temperature
        
    Returns
    -------
    None
    """
    
    sql = '''
          INSERT INTO weather_table(city_name, state_name, datetime, temperature)
          VALUES(?,?,?,?)
          '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

class City:
    '''
    A class to represent a city.
    
    Attributes
    ----------
    city_name : String
        name of city, not space nor capital sensitive
    state_name : String
        name of state, not space nor capital sensitive
    time_adjustment : int
        timezone adjustment in seconds
    
    Methods
    -------
    add_data()
        Request from an api, write raw data to raw_weather.txt, and insert into weather_table.
    '''
    
    def __init__(self, city_name, state_name, time_adjustment):
        '''
        Constructs all the necessary attributes for the city object.
        
        Parameters
        ----------
        city_name : String
            name of city, not space nor capital sensitive
        state_name : String
            name of state, not space nor capital sensitive
        time_adjustment : int
            timezone adjustment in seconds
        '''
        
        self.city_name = city_name
        self.state_name = state_name
        self.time_adjustment = time_adjustment
        self.url = f"https://api.openweathermap.org/data/2.5/weather?q={self.city_name}&appid=e9d2f749c8d866a3f1aa783791b18cdb&units=imperial"
    
    def add_data(self):
        '''
        Request from an api, write raw data to raw_weather.txt, and insert into weather_table.
        
        Returns
        -------
        None
        '''
        
        try:
            self.request = requests.get(self.url).json()
        except Exception as e:
            print('Error with the API request.')
        else:
            with open('raw_weather.txt', 'a') as f:
                f.write(json.dumps(self.request))

            self.weather_data = (self.request['name'],
                                 self.state_name,
                                 d.fromtimestamp(self.request['dt'] + self.time_adjustment),
                                 self.request['main']['temp'])
            create_project(conn, self.weather_data)

#austin = City("Austin", "Texas", central_time_adjustment)
#san_francisco = City("San Francisco", "California", pacific_time_adjustment)
#new_york = City("New York", "New York", eastern_time_adjustment)

with open('city_file.txt', "r") as f:
    cities = f.readlines()
    
cities_list = []
for line in cities:
    city_name, state_name, time_correction = line.split(",")
    print(city_name, state_name, time_correction)
    cities_list.append(City(city_name, state_name, time_correction))
    
while True:
    conn = create_connection(database)

    #austin.add_data()
    #san_francisco.add_data()
    #new_york.add_data()
    
    for city in cities_list:
        try:
            city.add_data()
        except Exception as e:
            print('Error with adding data.')
    time.sleep(300)

Austin Texas 10800

San Francisco California 3600

New York New York 14400
Error with adding data.
Error with adding data.
Error with adding data.


KeyboardInterrupt: 

In [21]:
with open('city_file.txt', "r") as f:
    cities = f.readlines()
    
cities_list = []
for line in cities:
    city_name, state_name, time_correction = line.split(",")
    print(city_name, state_name, time_correction)
    cities_list.append(City(city_name, state_name, time_correction))

for city in cities_list:
    try:
        city.add_data()
    except Exception as e:
        print('Error with adding data.')
            
print(json.dumps(cities_list[1]).json())

Austin Texas 10800

San Francisco California 3600

New York New York 14400
Error with adding data.
Error with adding data.
Error with adding data.


TypeError: Object of type City is not JSON serializable

In [27]:

print(austin)